In [1]:
from imports import *

# set columnns display format
pd.set_option('display.max_columns', None)
# default pandas decimal number display format
# pd.options.display.float_format = '{:20,.3f}'.format

In [2]:
# import raw, messy data
df = acquire.get_zillow_data()

In [3]:
# clean data
df = prepare.prep_zillow(df)

In [4]:
# scale and split the data
train, validate, test = prepare.split_scale(df)

In [9]:
# adding location clusters to dataframe
cols = ['county_Los Angeles', 'county_Orange', 'zip_code']
kmeans = KMeans(n_clusters=4)
kmeans.fit(train[cols])

train['location_clusters'] = kmeans.predict(train[cols])
validate['location_clusters'] = kmeans.predict(validate[cols])
test['location_clusters'] = kmeans.predict(test[cols])

In [11]:
# adding area clusters to dataframe - somewhat significant

cols =['total_sqft', 'lot_sqft', 'living_sqft']
kmeans = KMeans(n_clusters=3)
kmeans.fit(train[cols])

train['area_clusters'] = kmeans.predict(train[cols])
validate['area_clusters'] = kmeans.predict(validate[cols])
test['area_clusters'] = kmeans.predict(test[cols])

In [12]:
# adding size clusters to dataframe - not significant

cols = ['bedrooms', 'bathrooms', 'full_bath']
kmeans = KMeans(n_clusters=3)
kmeans.fit(train[cols])
train['size_clusters'] = kmeans.predict(train[cols])
validate['size_clusters'] = kmeans.predict(validate[cols])
test['size_clusters'] = kmeans.predict(test[cols])

In [ ]:
# adding value clusters to dataframe - not significant

cols = ['structure_value', 'assessed_value', 'land_value','taxamount']
kmeans = KMeans(n_clusters=3)
kmeans.fit(train[cols])
train['value_clusters'] = kmeans.predict(train[cols])
validate['value_clusters'] = kmeans.predict(validate[cols])
test['value_clusters'] = kmeans.predict(test[cols])